In [ ]:
import pathlib
import shutil
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pymedphys
import pymedphys._wlutz.bbpredict
import pymedphys._wlutz.pylinac
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findfield
import pymedphys._wlutz.findbb
import pymedphys._wlutz.reporting

In [ ]:
penumbra = 2
edge_lengths = [30, 30]
initial_rotation = 0
bb_diameter = 8

bb_predictor_tol = 0.2

pd.set_option("display.max_rows", 101)

In [ ]:
original_dir = pathlib.Path(r'S:\DataExchange\iViewDB')

In [ ]:
images_dir = original_dir.joinpath('img')

In [ ]:
most_recent_two_images = list(images_dir.glob('*.jpg'))[-2::]
most_recent_two_images

In [ ]:
home = pathlib.Path.home()

In [ ]:
decoded_paths = []

for image_path in most_recent_two_images:
    filename_jpg = image_path.name
    filename_png = f"{image_path.stem}.png"
    
    home_jpg_location = home.joinpath(filename_jpg)
    home_png_location = home.joinpath(filename_png)
    
    print(f"{image_path} --> {home_jpg_location}")
    print(f"{home_jpg_location} --> {home_png_location}")   
    
    shutil.copy2(image_path, home_jpg_location)

    !bash -c "~/bin/wavelet/jpeg /mnt/c/Users/sbiggs/{filename_jpg} /mnt/c/Users/sbiggs/{filename_jpg}"
    !bash -c "convert -depth 16 /mnt/c/Users/sbiggs/{filename_jpg} /mnt/c/Users/sbiggs/{filename_png}"
    
    home_jpg_location.unlink()
    
    decoded_paths.append(home_png_location)

In [ ]:
field_centres = []
bb_centres = []

for decoded_path in decoded_paths:

    x, y, img = pymedphys._wlutz.iview.iview_image_transform(decoded_path)

    field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)
    initial_centre = pymedphys._wlutz.findfield.get_centre_of_mass(x, y, img)

    field_centre, field_rotation = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(
        field, edge_lengths, penumbra, initial_centre, initial_rotation=initial_rotation, pylinac_tol=np.inf
    )   

    bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(
        field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation
    )
    
    field_centres.append(field_centre)
    bb_centres.append(bb_centre)
    
    print(field_centre)
    print(bb_centre)
    print(np.array(field_centre) - np.array(bb_centre))
    
    fig = pymedphys._wlutz.reporting.image_analysis_figure(
        x,
        y,
        img,
        bb_centre,
        field_centre,
        field_rotation,
        bb_diameter,
        edge_lengths,
        penumbra,
    )
    
    plt.show()

In [ ]:
np.array(field_centres) - np.array(bb_centres)

In [ ]:
np.mean(np.array(field_centres) - np.array(bb_centres), axis=0)